# PA 6 - Kevin Bacon Game
- Zach Fechko
- Last Updated: April 18, 2022
- Version 1.0
- Description: Uses an unweighted graph to implement the kevin bacon game, connecting actors to kevin bacon through movies they star in

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from rich.console import Console
import graphviz as gv
from collections import deque

In [2]:
test = dict()
with open("data/actorsTest.txt") as file:
    for line in file:
        (key, value) = line.split('|')
        test[key] = value

print(test)

{'Actor ID': 'Actor Name\n', '1': 'Kevin Bacon\n', '100': 'actor1\n', '200': 'actor2\n', '300': 'actor3\n', '400': 'actor4\n', '500': 'actor5\n', '600': 'actor6'}


In [8]:
movies = pd.read_csv('data/moviesTest.txt', sep='|', index_col=False)
movies

,Movie ID,Movie Name
0,10,movie1
1,20,movie2
2,30,movie3
3,40,movie4
4,50,movie5


In [9]:
actors = pd.read_csv('data/actorsTest.txt', sep='|')
actors

,Actor ID,Actor Name
0,1,Kevin Bacon
1,100,actor1
2,200,actor2
3,300,actor3
4,400,actor4
5,500,actor5
6,600,actor6


In [17]:
print('actor8' in actors)

False


Pseudo code
1. Movie-actors dictionary:
    - movie <-> [list of actors]
2. for each movie:
    - for actor 1 in list of actors
        - for actor 2 in list of actors
            - if actor1 != actor2:
                - add actor 1 to graph
                - add actor 2 to graph
                - add edge (movie) between them

In [10]:
movie_actor_pairs = pd.read_csv('data/movie-actorsTest.txt', sep='|')
movie_actor_pairs

,Movie ID,Actor ID
0,10,1
1,10,100
2,10,200
3,20,100
4,20,300
5,30,300
6,30,200
7,40,300
8,40,400
9,50,500


In [11]:
# This is for seeing how to look through the dataframe to find a certain value
for row in range(len(movie_actor_pairs)):
    if movie_actor_pairs.iloc[row]['Actor ID'] == 300:
        print(movie_actor_pairs.iloc[row]['Movie ID'])


20
30
40


In [13]:
class Vertex:
    def __init__(self, actor_ID, actor_name):
        self.actor_ID = actor_ID #this will take on the role of the key
        self.name = actor_name
        self.stars_in = [] #list of movie ids to connect actors
        self.connected_to = dict()

    def add_neighbor(self, neighbor, movie_id=0):
        self.connected_to[neighbor] = movie_id

class Graph:
    def __init__(self):
        self.vert_list = dict()
        self.num_vertices = 0

    def add_vertex(self, id, name, df):
        """
        create a vertex item using id and name (don't need to check because this will be inserted by interating through a df)
        iterate through the movie actor pairs
        if the movie has an actor id that matches the current actor
        append the movie ids to the stars_in list
        """
        self.num_vertices += 1
        new_vertex = Vertex(id, name)
        for row in range(len(df)): #this will be the movie-actor pair df
            if df.iloc[row]['Actor ID'] == new_vertex.actor_ID:
                new_vertex.stars_in.append(df.iloc[row]['Movie ID'])
        self.vert_list[id] = new_vertex
        return new_vertex

    def add_edge(self, actor1, actor2):
        """
        if actor1 and actor2 have a movie with the same id
        make an edge with the weight equal to that movie id
        """
        if actor1.actor_id not in self.vert_list:
            nv = self.add_vertex(actor1)
        if actor2.actor_id not in self.vert_list:
            nv = self.add_vertex(actor2)
        common_movie = find_common(actor1, actor2, movies) #need to change this so it sets the edge weight to the name of the movie, not the id
        if common_movie: #if the two actors have an edge they can use
            self.vert_list[actor1].add_neighbor(self.vert_list[actor2], common_movie)



In [15]:
def find_common(v1: Vertex, v2: Vertex, df: pd.DataFrame):
    """
    Takes 2 vertices and the movies dataframe as arguments, iterates through v1's stars_in list to see if it has any common movie ids with v2
    returns the movie title if it exists, returns 0 if it doesn't
    """
    for value in v1.stars_in:
        if value in v2.stars_in:
            return df.iloc[value]['Movie Name'] #return the movie name
    return None

## Main game flow 
- create a vertex for every actor in actors.txt
- add edges between all the vertices
- prompt the user for an actor